In [1]:
from interactigen import Interactigen
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv(dotenv_path='../scratch/dev.env')

model = ChatOpenAI(model="gpt-4-0125-preview")

interactigen = Interactigen(
    model=model,
)

phrase_utterances = interactigen.generate_phrase_utterances(
    base_phrase="someone might express appreciation to a bot",
    quantity=20,
)

phrase_utterances

['Thank you so much for your help!',
 'I really appreciate your assistance.',
 "You've been incredibly helpful, thanks!",
 'Thanks a lot for the support!',
 'Much obliged for your help.',
 "I'm so grateful for your assistance.",
 'Your support means a lot, thank you.',
 'Cheers for the help!',
 "You're a lifesaver, thanks!",
 "I can't thank you enough for your help.",
 'Thanks a bunch!',
 'Really appreciate it.',
 'Super helpful, thanks!',
 'Thanks for the support!',
 'Much obliged!',
 'So grateful, thanks!',
 'Support means a lot, thanks!',
 'Cheers!',
 "You're a lifesaver!",
 "Can't thank you enough!",
 'Thanks!',
 'Appreciated.',
 'So helpful!',
 'Thanks a lot!',
 'Obliged.',
 'Grateful.',
 'Supportive, thanks!',
 'Cheers!',
 'Lifesaver!',
 "Can't thank enough!",
 'Thank you so much for your help!',
 'I really appreciate your assistance.',
 "You've been incredibly helpful, thanks!",
 'Thanks a lot for the support!',
 'Much obliged for your help.',
 "I'm so grateful for your assistan

In [ ]:
from interactigen import Interactigen
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv(dotenv_path='../scratch/dev.env')

model = ChatGoogleGenerativeAI(model="gemini-pro")

interactigen = Interactigen(
    model=model,
)

phrase_utterances = interactigen.generate_phrase_utterances(
    base_phrase="someone might express appreciation to a bot",
    quantity=20,
)

phrase_utterances